<a href="https://colab.research.google.com/github/takoyakidath/image-analysis/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import cv2
import numpy as np

# 白いキャンバスを作成
img = np.ones((400, 600, 3), dtype=np.uint8) * 255

# 円
cv2.circle(img, (100, 100), 50, (0, 0, 0), -1)

# 三角形
pts_triangle = np.array([[250, 50], [200, 150], [300, 150]], np.int32)
cv2.fillPoly(img, [pts_triangle], (0, 0, 0))

# 四角形（長方形）
cv2.rectangle(img, (400, 50), (550, 150), (0, 0, 0), -1)

# ちょっと傾いた四角形（台形っぽく）
pts_quad = np.array([[150, 250], [250, 230], [270, 330], [130, 350]], np.int32)
cv2.fillPoly(img, [pts_quad], (0, 0, 0))

cv2.imwrite("shapes.png", img)

True

In [30]:
img = cv2.imread("shapes.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
_, thresh = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY_INV)

In [31]:
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [32]:
import numpy as np
import cv2

def classify_shape(contour):
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * peri, True)
    vertices = len(approx)
    area = cv2.contourArea(contour)
    if peri == 0:
        return "unknown"

    circularity = 4 * np.pi * area / (peri ** 2)

    if vertices == 3:
        return "triangle"
    elif vertices == 4:
        x, y, w, h = cv2.boundingRect(approx)
        aspect_ratio = w / float(h)
        return "square" if 0.9 < aspect_ratio < 1.1 else "rectangle"
    else:
        return "circle" if circularity > 0.8 else "polygon"

In [33]:
output = img.copy()
for cnt in contours:
    shape = classify_shape(cnt)
    M = cv2.moments(cnt)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        cX, cY = 0, 0

    cv2.drawContours(output, [cnt], -1, (0, 0, 255), 2)
    cv2.putText(output, shape, (cX - 40, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
cv2.imwrite("output_shapes.png", output)

True